# OpenAI Vision API (Preview

This notebook demonstrates how to use the OpenAI Vision API to determine if a product is in stock or out of stock. The API uses GPT-4 to analyze an image and provide a response. The API is currently in preview and is not available to the public.

__Note:__ The OpenAI Vision API is currently in preview and is not available to the public. You will need to have access to the API to use it.

__Note:__ Processing images with the OpenAI Vision API may incur additional costs. With the current pricing and the images tested in this notebook, the cost was approximately $0.01 per image.

In [ ]:
import json
import os
import pandas as pd
from openai import OpenAI

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)
def get_openai_vision_response(prompt_text: str, img_url: str):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=5,
    )
    
    return response.choices[0].message.content
    
def load_input_data():
    with open('../input.json', 'r') as file:
        json_data = json.load(file)
    return json_data

def color_cells(row):
    if row["In-Stock"] == "Yes" and row["Out-of-Stock"] == "No":
        return ["background-color: #296644"] * len(row)
    else:
        return ["background-color: #82323a"] * len(row)

## Execution

The code below will load the input data from the `input.json` file and then use the OpenAI Vision API to determine if the product is in stock or out of stock. The results will be displayed in a table with green cells indicating the product is in stock and red cells indicating the product is out of stock.

In [ ]:
data = load_input_data()

prompt = "Is the product available for purchase? Please answer 'Yes' or 'No' only."
df = pd.DataFrame(columns=["Source", "In-Stock", "Out-of-Stock"])

for i in range(0, len(data)):
    in_stock_img_url = data[i]['in_stock']
    in_stock_response = get_openai_vision_response(prompt, in_stock_img_url)

    out_of_stock_img_url = data[i]['out_of_stock']
    out_of_stock_response = get_openai_vision_response(prompt, out_of_stock_img_url)

    df.loc[i] = [data[i]["name"], in_stock_response, out_of_stock_response]

print(f"{prompt}")
df.style.apply(color_cells, axis=1)